# Product Review Embeddings
## Author: Luis Eduardo Ferro Diez <a href="mailto:luis.ferro1@correo.icesi.edu.co">luis.ferro1@correo.icesi.edu.co</a>

This notebook contains the exploratory analysis over the Amazon product review dataset which will serve as ground truth of the semantic representation of the product categories for which we want later to characterize the geographic zones.

### Resources
* Dataset: http://jmcauley.ucsd.edu/data/amazon/

We are going to use the product reviews (aggresively deduplicated data) and the metadata of the products to build the semantic space.

In [1]:
metadata_path = "/media/ohtar10/Adder-Storage/datasets/amazon_products/reviews/metadata.json.gz"

Using the reading instructions from the web page since the json objects are not strict and cause problems when reading from pandas.

Also, since the file is significantly big to the point it can't fit into memory, we need a mechanism that can read it in chunks.

In [58]:
import pandas as pd
import gzip
import json
from itertools import islice

def read_file(path, lines=100):
    f = gzip.open(path, 'r')
    for l in islice(f, lines):
        #yield json.dumps(eval(l))
        yield eval(l)
        
def generate_head_df(path, lines=100):
    df = {}
    for k, v in enumerate(read_file(path, lines)):
        df[k] = v
    return pd.DataFrame.from_dict(df, orient='index')

In [23]:
metadata_df = generate_head_df(metadata_path, 10000)
metadata_df.head()

,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,0001048791,{'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,0000143561,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"[[Movies & TV, Movies]]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN
2,0000037214,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,"[[Clothing, Shoes & Jewelry, Girls], [Clothing...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams
3,0000032069,NaN,http://ecx.images-amazon.com/images/I/51EzU6qu...,"[[Sports & Outdoors, Other Sports, Dance, Clot...",Adult Ballet Tutu Cheetah Pink,NaN,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi
4,0000031909,{'Toys & Games': 201847},http://ecx.images-amazon.com/images/I/41xBoP0F...,"[[Sports & Outdoors, Other Sports, Dance]]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,7.00,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown


We are interested in `asin`, `categories`, `title` and `description` columns, let's also check if we have null values there.

In [24]:
columns = ['asin', 'categories', 'title', 'description']
metadata_df = metadata_df[columns]
metadata_df.isnull().sum()

asin              0
categories        7
title             6
description    5326
dtype: int64

It seems we have several null values in description, but we can cope with that as we have the reviews. However, it seems we have null categories, this won't serve us well for our purpose so let's get rid of them. We must take care of not getting rid of the records that lack description.

In [25]:
metadata_df = metadata_df[metadata_df.categories.notnull()]
metadata_df.head()

,asin,categories,title,description
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN
1,0000143561,"[[Movies & TV, Movies]]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ..."
2,0000037214,"[[Clothing, Shoes & Jewelry, Girls], [Clothing...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN
3,0000032069,"[[Sports & Outdoors, Other Sports, Dance, Clot...",Adult Ballet Tutu Cheetah Pink,NaN
4,0000031909,"[[Sports & Outdoors, Other Sports, Dance]]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...


In [26]:
metadata_df.isnull().sum()

asin              0
categories        0
title             4
description    5322
dtype: int64

We observe we have matrices for categories, it would be better to faltten them for easier usage.

In [27]:
from itertools import chain

categories = list(chain.from_iterable(metadata_df.categories.values))
categories[:10]

[['Books'],
 ['Movies & TV', 'Movies'],
 ['Clothing, Shoes & Jewelry', 'Girls'],
 ['Clothing, Shoes & Jewelry',
  'Novelty, Costumes & More',
  'Costumes & Accessories',
  'More Accessories',
  'Kids & Baby'],
 ['Sports & Outdoors', 'Other Sports', 'Dance', 'Clothing', 'Girls', 'Skirts'],
 ['Sports & Outdoors', 'Other Sports', 'Dance'],
 ['Sports & Outdoors', 'Other Sports', 'Dance', 'Clothing', 'Girls', 'Skirts'],
 ['Movies & TV', 'Movies'],
 ['Books'],
 ['Sports & Outdoors', 'Other Sports', 'Dance']]

And flatten them and obtain the unique ones

In [28]:
flatten = lambda l: [item for sublist in l for item in sublist]

# Use set to eliminate duplicates
categories_flat = set(flatten(categories))
categories_flat

{'Active',
 'Active Skirts',
 'Alternative Rock',
 'Books',
 'Business & Office',
 'CDs & Vinyl',
 "Children's",
 "Children's Music",
 'Christian',
 'Clothing',
 'Clothing, Shoes & Jewelry',
 'Costumes & Accessories',
 'Dance',
 'General Accessories',
 'Girls',
 'Gospel',
 'Indie & Lo-Fi',
 'Indie Rock',
 'Instrument Accessories',
 'Jigsaw Puzzles',
 'Kids & Baby',
 'Lamps & Shades',
 'Lighting & Ceiling Fans',
 'More Accessories',
 'Movies',
 'Movies & TV',
 'Musical Instruments',
 'Novelty, Costumes & More',
 'Other Sports',
 'Pop & Contemporary',
 'Praise & Worship',
 'Puzzles',
 'Sheet Music Folders',
 'Skirts',
 'Software',
 'Sports & Outdoors',
 'TV',
 'Table Lamps',
 'Tools & Home Improvement',
 'Toys & Games',
 'Visualization & Presentation'}

Now let's extract the titles and descriptions for each category as separate fields (as arrays) so we can use them for later processing.

In [29]:
# Create a new field with a flattened version of the categories column for better computation
metadata_df['categories_flattened'] = metadata_df.categories.apply(lambda cat: list(chain.from_iterable(cat)))
metadata_df.head()

,asin,categories,title,description,categories_flattened
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,[Books]
1,0000143561,"[[Movies & TV, Movies]]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...","[Movies & TV, Movies]"
2,0000037214,"[[Clothing, Shoes & Jewelry, Girls], [Clothing...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,"[Clothing, Shoes & Jewelry, Girls, Clothing, S..."
3,0000032069,"[[Sports & Outdoors, Other Sports, Dance, Clot...",Adult Ballet Tutu Cheetah Pink,NaN,"[Sports & Outdoors, Other Sports, Dance, Cloth..."
4,0000031909,"[[Sports & Outdoors, Other Sports, Dance]]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[Sports & Outdoors, Other Sports, Dance]"


The above modification allow us to perform some searching via lambdas.

In [30]:
books = metadata_df.categories_flattened.apply(lambda cl: 'Books' in cl)
metadata_df[books]

,asin,categories,title,description,categories_flattened
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,[Books]
7,0001048775,[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,[Books]
9,0001048236,[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...",[Books]
10,0000401048,[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,[Books]
11,0001019880,[[Books]],Classic Soul Winner's New Testament Bible,NaN,[Books]
...,...,...,...,...,...
9995,0030547784,[[Books]],Holt McDougal Library: Individual Leveled Read...,NaN,[Books]
9996,0030542987,[[Books]],"Holt, Rinehart and Winston Algebra 1: Student ...",NaN,[Books]
9997,0030543843,[[Books]],Datasheets for LabBook - Holt Science and Tech...,NaN,[Books]
9998,0030500036,[[Books]],"Biology, Fourth Edition",NaN,[Books]


Let's define a function to facilitate this:

In [31]:
def get_products(df:pd.DataFrame, category:str, field='categories_flattened')-> pd.DataFrame:
    cat_indexes = df[field].apply(lambda cl: category in cl)
    return df[cat_indexes]

In [32]:
get_products(metadata_df, 'Movies')

,asin,categories,title,description,categories_flattened
1,0000143561,"[[Movies & TV, Movies]]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...","[Movies & TV, Movies]"
6,0000589012,"[[Movies & TV, Movies]]",Why Don't They Just Quit? DVD Roundtable Discu...,NaN,"[Movies & TV, Movies]"
28,0000695009,"[[Movies & TV, Movies]]",Understanding Seizures and Epilepsy DVD,NaN,"[Movies & TV, Movies]"
34,000107461X,"[[Movies & TV, Movies]]",Live in Houston [VHS],NaN,"[Movies & TV, Movies]"
36,0000143529,"[[Movies & TV, Movies]]",My Fair Pastry (Good Eats Vol. 9),Disc 1: Flour Power (Scones; Shortcakes; South...,"[Movies & TV, Movies]"
45,0000143502,"[[Movies & TV, Movies]]",Rise and Swine (Good Eats Vol. 7),Rise and Swine (Good Eats Vol. 7) includes bon...,"[Movies & TV, Movies]"
65,0000143588,"[[Movies & TV, Movies]]","Barefoot Contessa (with Ina Garten), Entertain...",Barefoot Contessa Volume 2: On these three dis...,"[Movies & TV, Movies]"
66,0001517791,"[[Movies & TV, Movies]]",Praise Aerobics [VHS],Praise Aerobics - A low-intensity/high-intesit...,"[Movies & TV, Movies]"
73,0001527665,"[[Movies & TV, Movies]]",Peace Child [VHS],NaN,"[Movies & TV, Movies]"
91,0001516035,"[[Movies & TV, Movies]]",Worship with Don Moen [VHS],Worship with Don Moen [VHS],"[Movies & TV, Movies]"


## Combine with product reviews
Now that we are able to obtain all the metadata for a particular category, let's now obtain the reviews per category. For a more interesting analysis, we are interested only in categories with ten or more reviews.

In [33]:
product_reviews_path = "/media/ohtar10/Adder-Storage/datasets/amazon_products/reviews/aggressive_dedup.json.gz"

In [34]:
pr_df = generate_head_df(product_reviews_path, 1000)
pr_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A00000262KYZUE4J55XGL,B003UYU16G,Steven N Elich,"[0, 0]",It is and does exactly what the description sa...,5.0,Does what it's supposed to do,1353456000,"11 21, 2012"
1,A000008615DZQRRI946FO,B005FYPK9C,mj waldon,"[0, 0]",I was sketchy at first about these but once yo...,5.0,great buy,1357603200,"01 8, 2013"
2,A00000922W28P2OCH6JSE,B000VEBG9Y,Gabriel Merrill,"[0, 0]",Very mobile product. Efficient. Easy to use; h...,3.0,Great product but needs a varmint guard.,1395619200,"03 24, 2014"
3,A00000922W28P2OCH6JSE,B001EJMS6K,Gabriel Merrill,"[0, 0]",Easy to use a mobile. If you're taller than 4f...,4.0,Great inexpensive product. Mounts easily and t...,1395619200,"03 24, 2014"
4,A00000922W28P2OCH6JSE,B003XJCNVO,Gabriel Merrill,"[0, 0]",Love this feeder. Heavy duty & capacity. Best ...,4.0,Great feeder. Would recommend for use for thos...,1395619200,"03 24, 2014"


We are interested only in what it was said about the product so we will only use: `asin`, `reviewText` and `summary`.

In [35]:
pr_columns = ['asin', 'summary', 'reviewText']
pr_df = pr_df[pr_columns]
pr_df.head()

,asin,summary,reviewText
0,B003UYU16G,Does what it's supposed to do,It is and does exactly what the description sa...
1,B005FYPK9C,great buy,I was sketchy at first about these but once yo...
2,B000VEBG9Y,Great product but needs a varmint guard.,Very mobile product. Efficient. Easy to use; h...
3,B001EJMS6K,Great inexpensive product. Mounts easily and t...,Easy to use a mobile. If you're taller than 4f...
4,B003XJCNVO,Great feeder. Would recommend for use for thos...,Love this feeder. Heavy duty & capacity. Best ...


In [36]:
reviews_df = metadata_df.join(pr_df.set_index('asin'), on='asin', how='inner')
reviews_df.head()

,asin,categories,title,description,categories_flattened,summary,reviewText
3607,000727405X,[[Books]],Cry Silent Tears: The heartbreaking survival s...,"Now thirty four years old, Joe Peters survived...",[Books],We need to know this,It really does happen that children are abused...
4201,0007444117,[[Books]],"Allegiant (Divergent, #3)",NaN,[Books],Allegiant a great ending to a great series.,Great book. Veronica Roth is extremely talente...


#### Note:
No idea what I was trying to do here....

In [37]:
from collections import defaultdict

categories_dict = defaultdict(dict)
for cat in categories_flat:
    cat_indexes = metadata_df.categories_flattened.apply(lambda clist: cat in clist)
    filtered = metadata_df[cat_indexes]
    categories_dict[cat] = {'titles': filtered.title.values, }

### Read the whole set in chunks
Now that we have the basic idea of how to pre-process the datasets, let's try to read the whole files in chunks.

In [55]:
def read_file_chunks(path, line_number, chunksize=1000):
    f = gzip.open(path, 'r')
    for index in range(0, line_number, chunksize):
        for l in islice(f, index, chunksize):
            yield json.dumps(eval(l))
            
def generate_df(path, chunksize=1000):
    df = {}
    for k, v in enumerate(read_file_chunks(path, lines)):
        df[k] = v
    return pd.DataFrame.from_dict(df, orient='index')

In [5]:
import gc
import gzip
from itertools import islice
import pandas as pd

def read_file(path, start=0, chunksize=1000):
    f = gzip.open(path, 'r')
    for l in islice(f, start, chunksize):
        yield eval(l)
        
def generate_df(path, fields, max_lines, chunksize=1000):
    df = {}
    for index in range(0, max_lines, chunksize):
        for k, v in enumerate(read_file(path, index, chunksize), start=index):
            obj = {key:value for key, value in v.items() if key in fields}
            df[k] = obj
            # release memory
            del v
        gc.collect()
    return pd.DataFrame.from_dict(df, orient='index')

In [6]:
%%time
metadata_fields = ['asin', 'categories', 'title', 'description']
metadata_full = generate_df(metadata_path, metadata_fields, 9430088, 10000)

CPU times: user 7h 1min 51s, sys: 4min 51s, total: 7h 6min 42s
Wall time: 7h 6min 43s


In [7]:
metadata_full.head()

,asin,categories,title,description
0,0001048791,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN
1,0000143561,"[[Movies & TV, Movies]]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ..."
2,0000037214,"[[Clothing, Shoes & Jewelry, Girls], [Clothing...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN
3,0000032069,"[[Sports & Outdoors, Other Sports, Dance, Clot...",Adult Ballet Tutu Cheetah Pink,NaN
4,0000031909,"[[Sports & Outdoors, Other Sports, Dance]]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...


In [9]:
metadata_full.count()

asin           10000
categories      9993
title           9994
description     4674
dtype: int64

### Warning!!!!
Do not run these lines too often, these are only for counting the lines in the files as are needed for reading the file in chunks.

In [52]:
def count_lines(path:str)->int:
    with gzip.open(path, 'r') as file:
        return sum(1 for line in file)

In [49]:
%%time
metadata_lines = count_lines(metadata_path)

CPU times: user 55.1 s, sys: 620 ms, total: 55.8 s
Wall time: 55.8 s


In [53]:
%%time
product_review_lines = count_lines(product_reviews_path)

CPU times: user 5min 28s, sys: 5.66 s, total: 5min 33s
Wall time: 5min 33s


In [54]:
# For reference
print(f"Metadata lines {metadata_lines}")
print(f"Product review lines {product_review_lines}")

Metadata lines 9430088
Product review lines 82456877


Metadata lines `9430088` Product review lines `82456877`